In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import regionmask
import matplotlib.pyplot as plt

/opt/anaconda3/envs/python-gis/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [10]:
import os
os.chdir('/Users/babak.jfard/projects/WNV/Codes')

In [168]:
# spi = xr.open_dataset('../data/raw/spi1_REGIONS_PRISM.nc')
spi = xr.open_dataset('../data/raw/spei1_REGIONS_PRISM.nc')

Now exploring the NetCDF file:

In [169]:
# The ID of the counties to extract from the netcdf file
IDs = pd.read_csv('../data/processed/lookup_revised.csv')

In [24]:
# Get a sample of how the final file should look like
spi_sample = pd.read_csv('../data/WNV_data_wrangling (1)/KSco_spi1_2022-03-15.csv')

In [25]:
spi_sample.head()

,County,month,year,spi1
0,Allen,1,1998,0.777578
1,Anderson,1,1998,0.167415
2,Atchison,1,1998,-0.801205
3,Barber,1,1998,0.030480
4,Barton,1,1998,-0.455410


In [170]:
# Getting the polygon IDs into a list
poly_IDs = list(IDs.polygon)

In [171]:
#IDs of all polygons to then index those that are needed
ncd_poly_IDs = spi.polygon.to_index().astype('int64').to_list()

In [172]:
# Now getting the indices of the polygons that we care about
poly_indices = [ncd_poly_IDs.index(i) for i in poly_IDs]

In [173]:
# ========= Cutting the dates
from datetime import datetime
start = datetime(1997,12, 30)
end = datetime(2022, 4, 1)


In [174]:
# now getting all the dates in spi and indexing the dates
ncd_time_IDs = spi.date.to_index()

In [175]:
dates = [(start <= i <= end) for i in ncd_time_IDs]

In [176]:
dates_indices = [i for i, x in enumerate(dates) if x]

In [177]:
# Final data
final_data = spi.data[dates_indices, poly_indices]

In [178]:
final_data

<xarray.DataArray 'data' (month: 291, polygon: 648)>
[188568 values with dtype=float32]
Coordinates:
  * polygon  (polygon) float32 1.296e+09 1.501e+09 ... 2.586e+09 3.36e+09
Dimensions without coordinates: month
Attributes:
    description:  Standardized Precipitation-Evapotranspiration Index 1 months
    units:        unitless

In [179]:
# Get column and row IDS
col_ids = [ncd_poly_IDs[id] for id in poly_indices]
# ncd_poly_IDs = spi.polygon.to_index().astype('int64').to_list()
time_labels = spi.date.to_index()[dates_indices]

In [180]:
final_data = final_data.values

In [181]:
data_to_save = pd.DataFrame(data=final_data,
index=time_labels, columns=col_ids)

In [182]:
data_to_save.head()

,1295778944,1501276032,1116729984,2937234944,2319075072,1645862016,1478552064,3703329024,2002216960,1654692992,...,1529528960,2896390912,2081657984,2178456064,2342443008,1671149056,2071234944,2368460032,2585875968,3360267008
month,,,,,,,,,,,,,,,,,,,,,
1998-01-15,0.782637,0.176496,-0.751577,-0.013616,-0.363692,0.997658,-0.385101,-0.068473,-0.136567,0.814598,...,-1.423877,0.406416,-0.239864,-1.484156,1.373400,1.308156,-1.337735,1.201282,-1.570901,-0.696835
1998-02-15,-1.372550,-0.962264,-0.104059,-0.431859,1.181718,-1.482892,0.817722,-0.858426,-0.747782,-1.145944,...,1.950785,1.384763,1.384797,-0.434528,0.551864,0.654425,0.441451,0.607840,1.101838,0.932361
1998-03-15,1.420949,1.448493,1.246696,1.633780,1.413923,1.479943,1.329778,1.456582,1.527683,2.032398,...,0.429708,0.528288,0.582141,0.432451,1.360007,0.153797,1.018625,1.925017,0.890947,1.322288
1998-04-15,0.536110,0.407083,-0.142372,-0.081246,0.078135,0.210045,0.138604,0.379958,0.208318,0.564290,...,-0.473062,-0.684313,-0.875267,-0.067546,-0.643862,-1.058104,-0.302078,-0.736931,-0.801289,-1.077066
1998-05-15,-3.261076,-2.603345,-2.085872,-0.932622,-1.692628,-2.278759,-2.207540,-1.899934,-2.094747,-1.269670,...,-1.967250,-2.008925,-1.982724,-1.909952,-1.688639,-1.983921,-2.015970,-2.044721,-2.231683,-2.170703


In [183]:
# data_to_save.to_csv('../data/processed/spi_initial.csv')
data_to_save.to_csv('../data/processed/spei_initial.csv')